In [ ]:
import pandas as pd
import numpy as np

import sys
sys.path.insert(1, '../framework/') # base location of the .py classes
from racetrack import *
rt = RACETrack()

from transformers import AutoTokenizer, RobertaForMaskedLM
import torch
import tensorflow as tf

In [ ]:
#
# Source:  https://en.wikipedia.org/wiki/Apollo_13
# ... four sentence summary
# ... chatgpt gave me five though :(
#
_text = """Apollo 13 (April 11–17, 1970) was the seventh crewed mission in the Apollo space program and the third meant to land on the Moon. The craft was launched from Kennedy Space Center on April 11, 1970, but the lunar landing was aborted after an oxygen tank in the service module (SM) failed two days into the mission. The crew instead looped around the Moon in a circumlunar trajectory and returned safely to Earth on April 17. The mission was commanded by Jim Lovell, with Jack Swigert as command module (CM) pilot and Fred Haise as Lunar Module (LM) pilot. Swigert was a late replacement for Ken Mattingly, who was grounded after exposure to rubella.
A routine stir of an oxygen tank ignited damaged wire insulation inside it, causing an explosion that vented the contents of both of the SM's oxygen tanks to space. Without oxygen, needed for breathing and for generating electric power, the SM's propulsion and life support systems could not operate. The CM's systems had to be shut down to conserve its remaining resources for reentry, forcing the crew to transfer to the LM as a lifeboat. With the lunar landing canceled, mission controllers worked to bring the crew home alive.
Although the LM was designed to support two men on the lunar surface for two days, Mission Control in Houston improvised new procedures so it could support three men for four days. The crew experienced great hardship, caused by limited power, a chilly and wet cabin and a shortage of potable water. There was a critical need to adapt the CM's cartridges for the carbon dioxide scrubber system to work in the LM; the crew and mission controllers were successful in improvising a solution. The astronauts' peril briefly renewed public interest in the Apollo program; tens of millions watched the splashdown in the South Pacific Ocean on television.
An investigative review board found fault with preflight testing of the oxygen tank and Teflon being placed inside it. The board recommended changes, including minimizing the use of potentially combustible items inside the tank; this was done for Apollo 14. The story of Apollo 13 has been dramatized several times, most notably in the 1995 film Apollo 13 based on Lost Moon, the 1994 memoir co-authored by Lovell – and an episode of the 1998 miniseries From the Earth to the Moon."""
# model, tokenizer, device = rt.__textTrainRoBERTaModel__(_text)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("roberta-base")
model = RobertaForMaskedLM.from_pretrained("roberta-base")

In [ ]:
_commented_out = '''
inputs = tokenizer(" Apollo 13 was the <mask> crewed mission in the Apollo space program and the third meant to land on the Moon. ", return_tensors="pt")
with torch.no_grad():
    output = model(**inputs)
    logits = output.logits
mask_token_index = (inputs.input_ids == tokenizer.mask_token_id)[0].nonzero(as_tuple=True)[0]
predicted_token_id = logits[0, mask_token_index].argmax(axis=-1)
tokenizer.decode(predicted_token_id)
'''

In [ ]:
_commented_out = '''
top_k_indices = tf.math.top_k(logits.detach().numpy(), 10).indices[0].numpy()
# tokenizer.decode(top_k_indices[mask_token_index])
for x in top_k_indices[mask_token_index]:
    print(tokenizer.decode(x))
'''

In [ ]:
_commented_out = '''
labels = tokenizer(" Apollo 13 was the seventh crewed mission in the Apollo space program and the third meant to land on the Moon. ", return_tensors="pt")["input_ids"]
labels = torch.where(inputs.input_ids == tokenizer.mask_token_id, labels, -100)
outputs = model(**inputs, labels=labels)
round(outputs.loss.item(), 2)
'''

In [ ]:
_commented_out = '''
token_i = 5
_sort   = []
for i in range(len(outputs[1][0][token_i])):
    _sort.append((outputs[1][0][token_i][i], i, tokenizer.decode(i)))
sorted(_sort, reverse=True)
'''

In [ ]:
_commented_out = '''
histogram  = outputs[1][0][token_i].histogram(bins=20)
word_score = float(outputs[1][0][token_i][labels[0][token_i]])
bg         = {'word':[(word_score,0),(word_score,10000)]}
_df        = pd.DataFrame({'w':histogram.hist.detach().numpy(), 'x':histogram.bin_edges[1:].detach().numpy(), 'gb':np.ones(len(histogram.hist))})
rt.displaySVG(rt.xy(_df, x_field='x', y_field='w', count_by='w', dot_size='small', line_groupby_field='gb', bg_shape_lu=bg, w=256,h=96))
'''

In [ ]:
svgs = []
_sentence = 'Flight 13 landed at Kennedy Space Center.'
_sentence = ' ' + _sentence.strip() + ' '
_inputs   = tokenizer(_sentence, return_tensors="pt")
for i in range(3,5): # range(1,len(_inputs['input_ids'][0])-2):
    _token                  = tokenizer.decode(_inputs['input_ids'][0][i])
    _token_stripped_lowered = _token.strip().lower()
    _before = ' ' if _token[0]  == ' ' else ''
    _after  = ' ' if _token[-1] == ' ' else ''
    _sentence_w_mask = tokenizer.decode(_inputs['input_ids'][0][1:i]) + _before + '<mask>' + _after + tokenizer.decode(_inputs['input_ids'][0][i+1:-1])
    _inputs_w_mask   = tokenizer(_sentence_w_mask, return_tensors="pt")
    with torch.no_grad():
        _output = model(**_inputs_w_mask)
        _logits = _output.logits
        in_top_20,ith = None, 0
        top_k_indices = tf.math.top_k(_logits.detach().numpy(), 20).indices[0].numpy()
        for x in top_k_indices[i]:
            _str = tokenizer.decode(x).strip().lower()
            if _str == _token_stripped_lowered and in_top_20 is None:
                in_top_20 = ith
            ith += 1
        mask_token_index   = (_inputs_w_mask.input_ids == tokenizer.mask_token_id)[0].nonzero(as_tuple=True)[0]
        predicted_token_id = _logits[0, mask_token_index].argmax(axis=-1)
        predicted_word     = tokenizer.decode(predicted_token_id)
        print(f'{_sentence_w_mask:70}\t{predicted_word:30}\t{_token:30}\t{in_top_20}\t| {len(_inputs_w_mask.input_ids[0])}\t{len(_inputs.input_ids[0])}\t| {i}\t{mask_token_index}')

        labels  = tokenizer(_sentence, return_tensors="pt")["input_ids"]
        labels  = torch.where(_inputs.input_ids==tokenizer.mask_token_id, labels, -100)
        outputs = model(**_inputs, labels=labels)
        word_score = float(outputs[1][0][i][labels[0][i]])
        print(word_score)

        histogram  = outputs[1][0][i].histogram(bins=100)
        bg         = {'word':[(word_score,0),(word_score,100000)]}
        _df        = pd.DataFrame({'w':histogram.hist.detach().numpy(), _token:histogram.bin_edges[1:].detach().numpy(), 'gb':np.ones(len(histogram.hist))})
        svgs.append(rt.xy(_df, x_field=_token, y_field='w', count_by='w', dot_size=None, line_groupby_field='gb', bg_shape_lu=bg, w=256,h=96))

In [ ]:
rt.displaySVG(rt.tile(svgs,horz=False))